# Providers and products

In [44]:
from eodag import EODataAccessGateway
dag = EODataAccessGateway()

## Providers available

The method [available_providers()](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway.available_providers) returns a list of the pre-configured providers.

In [45]:
available_providers = dag.available_providers()
available_providers

['astraea_eod',
 'aws_eos',
 'creodias',
 'earth_search',
 'mundi',
 'onda',
 'peps',
 'sobloo',
 'theia',
 'usgs',
 'usgs_satapi_aws',
 'wekeo']

In [46]:
print(f"eodag has {len(available_providers)} providers already configured.")

eodag has 12 providers already configured.


It can take a product type as an argument and will return the providers known to `eodag` that offer this product.

In [47]:
dag.available_providers("S2_MSI_L1C")

['astraea_eod',
 'aws_eos',
 'creodias',
 'earth_search',
 'mundi',
 'onda',
 'peps',
 'sobloo',
 'wekeo']

## Product types available

The method [list_product_types()](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway.list_product_types) returns a dictionary that represents `eodag`'s internal product type catalog.

In [51]:
catalog = dag.list_product_types()

In [52]:
catalog[0]

{'ID': 'CBERS4_AWFI_L2',
 'abstract': 'CBERS-4 AWFI camera Level-2 product. System corrected images, expect some\ntranslation error.\n',
 'instrument': 'AWFI',
 'platform': 'China-Brazil Earth Resources Satellite',
 'platformSerialIdentifier': 'CBERS-4',
 'processingLevel': 'L2',
 'sensorType': 'OPTICAL',
 'license': 'proprietary',
 'missionStartDate': '2014-12-07T00:00:00Z',
 'title': 'CBERS-4 AWFI Level-2'}

One of these product types has for ID `GENERIC_PRODUCT_TYPE`. It is not meant to be used directly, instead, its goal is to act as a template for searching for product types not listed in `eodag`'s catalog. See [the search page](4_search.ipynb#product-type) for more information.

In [62]:
products_id = [p["ID"] for p in catalog if p["ID"] != "GENERIC_PRODUCT_TYPE"]
products_id

['CBERS4_AWFI_L2',
 'CBERS4_AWFI_L4',
 'CBERS4_MUX_L2',
 'CBERS4_MUX_L4',
 'CBERS4_PAN10M_L2',
 'CBERS4_PAN10M_L4',
 'CBERS4_PAN5M_L2',
 'CBERS4_PAN5M_L4',
 'L57_REFLECTANCE',
 'L8_OLI_TIRS_C1L1',
 'L8_REFLECTANCE',
 'LANDSAT_C2L1',
 'LANDSAT_C2L2ALB_BT',
 'LANDSAT_C2L2ALB_SR',
 'LANDSAT_C2L2ALB_ST',
 'LANDSAT_C2L2ALB_TA',
 'LANDSAT_C2L2_SR',
 'LANDSAT_C2L2_ST',
 'MODIS_MCD43A4',
 'NAIP',
 'OSO',
 'PLD_BUNDLE',
 'PLD_PAN',
 'PLD_PANSHARPENED',
 'PLD_XS',
 'S1_SAR_GRD',
 'S1_SAR_OCN',
 'S1_SAR_RAW',
 'S1_SAR_SLC',
 'S2_MSI_L1C',
 'S2_MSI_L2A',
 'S2_MSI_L2A_MAJA',
 'S2_MSI_L2B_MAJA_SNOW',
 'S2_MSI_L2B_MAJA_WATER',
 'S2_MSI_L3A_WASP',
 'S3_EFR',
 'S3_ERR',
 'S3_LAN',
 'S3_OLCI_L2LFR',
 'S3_OLCI_L2LRR',
 'S3_SLSTR_L1RBT',
 'S3_SLSTR_L2LST',
 'S3_SRA',
 'S3_SRA_A_BS',
 'S3_SRA_BS',
 'S3_WAT',
 'SPOT5_SPIRIT',
 'SPOT_SWH',
 'SPOT_SWH_OLD',
 'VENUS_L1C',
 'VENUS_L2A_MAJA',
 'VENUS_L3A_MAJA']

In [68]:
print(f"EODAG has {len(products_id)} product types stored in its internal catalog.")

EODAG has 52 product types stored in its internal catalog.


The method can take a provider name as an argument and will return the product types known to `eodag` that are offered by this provider.

In [64]:
peps_products = dag.list_product_types("peps")
[p["ID"] for p in peps_products if p["ID"] != "GENERIC_PRODUCT_TYPE"]

['S1_SAR_GRD',
 'S1_SAR_OCN',
 'S1_SAR_SLC',
 'S2_MSI_L1C',
 'S2_MSI_L2A',
 'S3_EFR',
 'S3_ERR',
 'S3_LAN',
 'S3_OLCI_L2LFR',
 'S3_OLCI_L2LRR',
 'S3_SLSTR_L1RBT',
 'S3_SLSTR_L2LST']

## Combine these two methods

These two methods can be combined to find which product type is the most common in `eodag`'s catalog among all the providers.

In [65]:
availability_per_product = []
for product in products_id:
    providers = dag.available_providers(product)
    availability_per_product.append((product, len(providers)))
availability_per_product = sorted(availability_per_product, key=lambda x: x[1], reverse=True)
most_common_p_type, nb_providers = availability_per_product[0]
print(f"The most common product type is '{most_common_p_type}' with {nb_providers} providers offering it.")

('S2_MSI_L1C', 9)

These can be also used to find out which provider (as configured by `eodag`) offers the hights number of different product types.

In [71]:
availability_per_provider = []
for provider in dag.available_providers():
    provider_products_id = [
        p["ID"]
        for p in dag.list_product_types(provider)
        if p["ID"] != "GENERIC_PRODUCT_TYPE"
    ]
    availability_per_provider.append(
        (provider, len(provider_products_id))
    )
availability_per_provider = sorted(availability_per_provider, key=lambda x: x[1], reverse=True)
provider, nb_p_types = availability_per_provider[0]
print(f"The provider with the largest number of product types is '{provider}' with {nb_p_types}.")

The provider with the largest number of product types is 'onda' with 18.
